In [1]:
import os
os.chdir("../")

In [3]:
%pwd

'c:\\Users\\odz-2\\Desktop\\learning\\medical-AI-chatbot\\Medical-Chatbot-Example'

In [4]:
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

c:\Users\odz-2\anaconda3\envs\medibot_gq\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
#Extract text from PDF files
def load_pdf_files(data):
    loader = DirectoryLoader(
        data,
        glob="*.pdf",
        loader_cls=PyPDFLoader
    )

    documents = loader.load()
    return documents

In [ ]:
extracted_data = load_pdf_files("data")

In [6]:
from typing import List
from langchain_core.documents import Document

def filter_to_minimal_docs(docs: List[Document]) -> List[Document]:
    """
    Given a list of Document objects, return a new list of Document objects
    containing only 'source' in metadata and the original page_content.
    """
    minimal_docs: List[Document] = []
    for doc in docs:
        src = doc.metadata.get("source")
        minimal_docs.append(
            Document(
                page_content = doc.page_content,
                metadata={"source": src}
            )
        )
    return minimal_docs

In [7]:
minimal_docs = filter_to_minimal_docs(extracted_data)

In [8]:
# Split the documents into smaller chunks
def text_split(minimal_docs):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500, #500 tokens equal to one chunk
        chunk_overlap=20, #understand the context with this overlap
    )
    texts_chunks = text_splitter.split_documents(minimal_docs)
    return texts_chunks

In [9]:
texts_chunks = text_split(minimal_docs)
print(f"Number of chunks: {len(texts_chunks)}")

Number of chunks: 5859


In [7]:
#Embedding model: HuggingFaceEmbeddings -> SentenceTransformer
from langchain_huggingface import HuggingFaceEmbeddings

def download_embeddings():

    model_name="all-MiniLM-L6-v2"
    embeddings = HuggingFaceEmbeddings(
        model_name=model_name
    )
    return embeddings

embedding = download_embeddings()


In [12]:
vector = embedding.embed_query("Hello World")


In [12]:
print("Vector length: ", len(vector))

Vector length:  384


In [8]:
from dotenv import load_dotenv #importing the load_dotenv function
import os
load_dotenv()


True

In [9]:
#Groq version API Keys
PINECONE_API_KEY= os.getenv("PINECONE_API_KEY")
GROQ_API_KEY= os.getenv("GROQ_API_KEY")


os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY #saving it as an environment variable
os.environ["GROQ_API_KEY"] = GROQ_API_KEY

In [10]:
from pinecone import Pinecone
pinecone_api_key = PINECONE_API_KEY

pc = Pinecone(api_key=pinecone_api_key)

In [11]:
from pinecone import ServerlessSpec

index_name = "medical-chatbot"

index = pc.Index(index_name)

In [18]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=texts_chunks,
    embedding=embedding,
    index_name=index_name
)

#it will take all the text chunks and will use the embedding model
# to convert them into vector datatbase
# to store them in Pinecone vector database 

In [12]:
from langchain_pinecone import PineconeVectorStore
#Embed each chunk and upsert ther embeddings into your Pinecone index
docsearch = PineconeVectorStore.from_existing_index(
    embedding=embedding,
    index_name=index_name
)

In [13]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})
#creting the retriever to bring the top 3 most relevant documents

In [14]:
retrieved_docs = retriever.invoke("What is Acne?")
retrieved_docs
#it brings three most relevant documents

[Document(id='aa9b5c90-4b5c-4b30-94ac-1c03a23f9845', metadata={'source': 'data\\Medical_book.pdf'}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(id='59656227-40e7-4e2b-a569-c93199c81c25', metadata={'source': 'data\\Medical_book.pdf'}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(id='54287b06-9e36-44df-8219-6cca65fb08f0', metadata={'source': 'data\\Medical_book.pdf'}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 2 25\nAcne\nAcne vulgaris affecting a woman’s face. Acne is the general\nname given to a skin disorder in which the sebaceous\nglands become inflamed. (Photograph by Biophoto Associ-\nates, Photo Researchers, Inc. Reproduced by permission.)\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 25')]

In [15]:
from langchain_groq import ChatGroq

chatModel = ChatGroq(model="llama-3.1-8b-instant")


In [16]:
from langchain_classic.chains import create_retrieval_chain
from langchain_classic.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

In [17]:
#prompt template
system_prompt = (
    "You are a medical assitant for question-answering tasks."
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

#creating the prompt template and giving the system and user roles
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("user", "{input}"),
    ]

)

In [18]:
#creating a chain to combine the documents using 'stuff' method
question_answer_chain = create_stuff_documents_chain(chatModel, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)


In [ ]:
#we can ask any kind of question
response = rag_chain.invoke({"input": "What is Acromegaly and gigantism?"})
print(response["answer"])
#print(response["context"])


Acromegaly is a disorder caused by the abnormal release of a chemical from the pituitary gland, leading to increased growth in bone and soft tissue, as well as various body disturbances. This results from an excess of growth hormone production. Gigantism is a similar condition that occurs in children before the bones have stopped growing.
[Document(id='38402036-b092-4d57-9a05-1de5d598e786', metadata={'source': 'data\\Medical_book.pdf'}, page_content='Whitehouse Station, NJ: Merck Research Laboratories,\n1997.\nLarsen, D. E., ed. Mayo Clinic Family Health Book.New York:\nWilliam Morrow and Co., Inc., 1996.\nJohn T. Lohr, PhD\nAcromegaly and gigantism\nDefinition\nAcromegaly is a disorder in which the abnormal\nrelease of a particular chemical from the pituitary gland\nin the brain causes increased growth in bone and soft tis-\nsue, as well as a variety of other disturbances throughout\nthe body. This chemical released from the pituitary gland'), Document(id='408761ef-c24d-4926-9171-797d

In [ ]:
response = rag_chain.invoke({"input": "What is the treatment for Acne?"})
print(response["answer"])
#print(response["context"])

## RAGAS Evaluation

In [19]:
import os
import pandas as pd
from datasets import Dataset
from dotenv import load_dotenv

from langchain_classic.chains import create_retrieval_chain
from langchain_classic.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

from langchain_pinecone import PineconeVectorStore
from langchain_huggingface import HuggingFaceEmbeddings

#RAGAS
from ragas import evaluate
from ragas.metrics import faithfulness, answer_relevancy 

load_dotenv()
RAGAS_JUDGE_GQ_API_KEY= os.getenv("RAGAS_JUDGE_GQ_API_KEY")
os.environ["RAGAS_JUDGE_GQ_API_KEY"] = RAGAS_JUDGE_GQ_API_KEY

C:\Users\odz-2\AppData\Local\Temp\ipykernel_45460\4070111154.py:15: DeprecationWarning: Importing faithfulness from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import faithfulness
  from ragas.metrics import faithfulness, answer_relevancy
C:\Users\odz-2\AppData\Local\Temp\ipykernel_45460\4070111154.py:15: DeprecationWarning: Importing answer_relevancy from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import answer_relevancy
  from ragas.metrics import faithfulness, answer_relevancy


In [20]:
#Test Questions Section

test_questions = [
    "What are the primary symptoms of Asthma?",
    "What defines Hypertension (High Blood Pressure)?",
    "What are the common causes of Iron Deficiency Anemia?",
    "How is Type 2 Diabetes primarily characterized?",
    "What are the early warning signs of Alzheimer's Disease?"
]

ground_truths = [
    "Asthma symptoms include wheezing, shortness of breath, chest tightness, and coughing.",
    "Hypertension is defined as having a blood pressure reading consistently at or above 140/90 mmHg.",
    "Iron deficiency anemia is caused by a lack of iron in the body due to blood loss or poor diet.",
    "Type 2 diabetes is characterized by insulin resistance and high blood sugar levels.",
    "Early signs of Alzheimer's include memory loss, confusion with time or place, and trouble finding words."
]

records = []

for question, ground_truth in zip(test_questions, ground_truths):
    response = rag_chain.invoke({"input": question})

    records.append({
        "question": question,
        "answer": response["answer"],
        "contexts": [doc.page_content for doc in response["context"]],
        "ground_truth": ground_truth,
    })

dataset = Dataset.from_pandas(pd.DataFrame(records))

In [21]:
from ragas.run_config import RunConfig
from langchain_google_genai import ChatGoogleGenerativeAI, HarmBlockThreshold, HarmCategory

#judge LLM
judge_llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash-lite",
    temperature=0,
    timeout=60,
    google_api_key=RAGAS_JUDGE_GQ_API_KEY,
    safety_settings={
        HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
    }
)

my_run_config = RunConfig(
    max_workers=1,
    timeout=120
)

results = evaluate(
    dataset,
    metrics=[faithfulness, answer_relevancy],
    llm=judge_llm,
    embeddings=embedding,
    run_config=my_run_config
)


Evaluating: 100%|██████████| 10/10 [01:15<00:00,  7.51s/it]


In [22]:
df_res = results.to_pandas()
df_res.to_csv("ragas_gq.csv", index=False)
df_res.head()

,user_input,retrieved_contexts,response,reference,faithfulness,answer_relevancy
0,What are the primary symptoms of Asthma?,"[patients, and they may sometimes be the sole ...",The primary symptoms of asthma include narrowe...,"Asthma symptoms include wheezing, shortness of...",0.857143,0.970518
1,What defines Hypertension (High Blood Pressure)?,"[(BPH), a condition that affects men and is ch...","Hypertension, or high blood pressure, is defin...",Hypertension is defined as having a blood pres...,1.000000,0.804440
2,What are the common causes of Iron Deficiency ...,[them are rare.\nIRON DEFICIENCY ANEMIA. Iron ...,Heavy bleeding causes significant iron loss. S...,Iron deficiency anemia is caused by a lack of ...,1.000000,0.986294
3,How is Type 2 Diabetes primarily characterized?,[maturity onset or non insulin-dependent. Type...,Type 2 Diabetes is primarily characterized by ...,Type 2 diabetes is characterized by insulin re...,0.500000,0.892521
4,What are the early warning signs of Alzheimer'...,"[severe problems with eating, communicating, a...",The early warning signs of Alzheimer's Disease...,Early signs of Alzheimer's include memory loss...,1.000000,0.935794
